## 문장 쌍 분류 모델

### 각종 설정

- 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)  

Mounted at /gdrive


- 의존성 패키지 설치

In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### 모델 환경 설정
모델 하이퍼 파라미터(hyper parameter)와 저장 위치 등 설정 정보를 선언

In [3]:
from torch.cuda import is_available
import torch 
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_task_name="pair-classification",
    downstream_corpus_name="klue-nli",
    downstream_model_dir="/gdrive/MyDrive/nlpbook/checkpoint-paircls1",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=1,
    #epochs=5,
    #tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

#### 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정

In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


#### 로거 설정
메세지 출력 등을 위한 logger를 설정

In [5]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoint-paircls1', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoint-paircls1', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_siz

### 말뭉치 내려받기

- 실습에 사용할 말뭉치(KLUE-NLI)를 다운로드
- 코랩 환경 로컬의 root_dir(/content/Korpora)이하에 저장

In [6]:
nlpbook.download_downstream_dataset(args)

INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!


### 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

### 데이터 전처리

#### 학습 데이터 구축

- ClassificationDataset: 각 instance를 포함하는 데이터셋 역할을 수행
  * 이 클래스는 KlueNLICorpus와 토크나이저를 품고 있음 
  * KlueNLICorpus는 JSON 파일 형식의 KLUE-NLI 데이터를 “문장(전제+가설) + 레이블(참,거짓,중립)” 형태로 읽어들이는 역할
  * KlueNLICorpus가 넘겨준 문장(전제, 가설)을 모델이 학습할 수 있는 형태(ClassificationFeatures)로 가공
  * 다시 말해 전제와 가설 2개 문장을 각각 토큰화하고 이를 인덱스로 변환하는 한편, 레이블 역시 정수(integer)로 바꿔주는 역할(entailment: 0, contradiction: 1, neutral: 2)

In [8]:
from ratsnlp.nlpbook.classification import ClassificationDataset
from ratsnlp.nlpbook.paircls import KlueNLICorpus
corpus = KlueNLICorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_train_BertTokenizer_64_klue-nli_pair-classification [took 1.891 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_train_BertTokenizer_64_klue-nli_pair-classification [took 1.891 s]


- input_ids: '[CLS] 전제 [SEP] 가설 [SEP]'를 토큰화한 뒤 인덱싱한 결과
  * 인덱스로 변환된 토큰 시퀀스
  * 패딩 토큰([PAD])의 인덱스에 해당하는 0이 많이 붙어 있음
  * 분석 대상 문장의 토큰 길이가 max_seq_length(64)보다 짧기 때문
  * 이보다 긴 문장일 경우 64로 줄임

- attention_mask: 해당 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타냄

- token_type_ids: 세그먼트(segment) 정보
  * '[CLS] 전제 [SEP]'에 해당하는 첫 번째 세그먼트: 0
  * '가설 [SEP]'에 해당하는 두 번째 세그먼트: 1
  * 나머지 패딩에 속하는 세 번째 세그먼트: 0

- label: 정수로 변환된 레이블 정보



In [9]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=1)

- 데이터 로더: ClassificationDataset 클래스가 들고 있는 전체 인스턴스 가운데 배치 크기(args의 batch_size)만큼을 뽑아 배치 형태로 가공(nlpbook.data_collator)하는 역할을 수행
  * 데이터 로더는 배치를 만들 때 ClassificationDataset이 들고 있는 전체 인스턴스 가운데 batch_size 갯수만큼을 비복원(replacement=False) 랜덤 추출(Random Sampler)

- sampler: 샘플링 방식을 정의 

- collate_fn: 이렇게 뽑힌 인스턴스를 배치로 만드는 역할을 하는 함수
  * nlpbook.data_collator: 같은 배치에서 인스턴스가 여럿일 때 이를 input_ids, attention_mask 등 종류별로 모으고 파이토치가 요구하는 자료형인 텐서 형태로 바꾸는 역할을 수행

In [10]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,  #배치 크기로 채우지 못한 마지막 불완전 배치의 사용 여부
    num_workers=args.cpu_workers,
)

- 전체 코드

In [ ]:
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset
corpus = KlueNLICorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,  #배치 크기로 채우지 못한 마지막 불완전 배치의 사용 여부
    num_workers=args.cpu_workers,
)

#### 평가용 데이터 구축

- 학습용 데이터 로더와 달리 평가용 데이터 로더는 SequentialSampler를 사용
- SequentialSampler는 batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할
- 학습 때 배치 구성은 랜덤으로 하는 것이 좋으나 평가할 때는 평가용 데이터 전체를 사용하므로 굳이 랜덤으로 구성할 이유가 없기 때문에 SequentialSampler를 사용

In [11]:
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset
corpus = KlueNLICorpus()
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

from torch.utils.data import DataLoader, SequentialSampler
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,  #배치 크기로 채우지 못한 마지막 불완전 배치의 사용 여부
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_test_BertTokenizer_64_klue-nli_pair-classification [took 0.077 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_test_BertTokenizer_64_klue-nli_pair-classification [took 0.077 s]


### 모델 초기화
- 프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화
- BertForSequenceClassification: 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스


In [12]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

### 모델 학습

- Task 정의
  * 파이토치 라이트닝이 제공하는 LightningModule 클래스를 상속받아 태스크를 정의
  * 위에서 만든 학습 설정(args)과 준비한 모델(model)을 ClassificationTask에 주입
  * ClassificationTask에는 옵티마이저(optimizer), 러닝 레이트 스케줄러(learning rate scheduler)가 정의되어 있음
  * 옵티마이저는 아담(Adam), 러닝 레이트 스케줄러는 ExponentialLR을 사용

In [13]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

- 트레이너 정의
  * 파이토치 라이트닝 라이브러리의 도움을 받아 GPU/TPU 설정, 로그 및 체크포인트 등의 설정을 알아서 해줌

In [14]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


- 학습
  * 준비한 데이터와 모델로 학습을 시작
  * 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치("/gdrive/MyDrive/nlpbook/checkpoint-paircls")에 저장

In [15]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.683   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]